In [106]:
import pandas as pd


esaa_data['Median HH Income'] = pd.to_numeric(esaa_data['Median HH Income'].astype(str).str.replace(',', '').str.replace('$', ''), errors='coerce')

racial_columns = [
    'Total:!!American Indian and Alaska Native alone',
    'Total:!!Asian alone',
    'Total:!!Black or African American alone',
    'Total:!!Native Hawaiian and Other Pacific Islander alone',
    'Total:!!White alone',
    'Total:!!Two or More Races',
    'Total:!!Some Other Race alone'
]

for col in racial_columns:
    esaa_data[col] = pd.to_numeric(esaa_data[col].astype(str), errors='coerce')


esaa_data['Total Population'] = esaa_data[racial_columns].sum(axis=1)


for col in racial_columns:
    esaa_data[f'Proportion of {col}'] = esaa_data[col] / esaa_data['Total Population']

# Step 5:filter missing data
cleaned_data = esaa_data.dropna(subset=['Median HH Income'] + [f'Proportion of {col}' for col in racial_columns])

# Calculate correlations
correlation_results = []
for col in racial_columns:
    correlation = cleaned_data['Median HH Income'].corr(cleaned_data[f'Proportion of {col}'])
    correlation_results.append(correlation)

# organize correlations 
correlation_data = {
    'Metric': ['Median HH Income'] + [f'Proportion of {col}' for col in racial_columns],
    'Correlation': [1.0] + correlation_results  # 1.0 for self-correlation
}

# put correlations in a table
correlation_table = pd.DataFrame(correlation_data)

# Display the correlation table
print(correlation_table)

 

                                              Metric  Correlation
0                                   Median HH Income     1.000000
1  Proportion of Total:!!American Indian and Alas...    -0.380524
2                  Proportion of Total:!!Asian alone    -0.122080
3  Proportion of Total:!!Black or African America...    -0.459527
4  Proportion of Total:!!Native Hawaiian and Othe...    -0.029617
5                  Proportion of Total:!!White alone     0.579056
6            Proportion of Total:!!Two or More Races     0.110975
7        Proportion of Total:!!Some Other Race alone    -0.412074


In [108]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQGyiwDDSUyeKtfJtH_L5TPEedSBOifOcI8gHUWBP0ufMIK3kT41cvft4wnA18a_BSWeKBYpHzoaTBB/pub?output=csv'
data = pd.read_csv(file_path, skiprows=2)

# Rename columns 
data.columns = ["Grade", "Program Code", "Program Name", "School Name", "Requests per seat", 
                "Total Requests", "Total Assigned", "Total Seats"]

# Convert to numeric values
data['Total Seats'] = pd.to_numeric(data['Total Seats'], errors='coerce')
data['Requests per seat'] = pd.to_numeric(data['Requests per seat'], errors='coerce')

# Filter data for only Kindergarten 
kindergarten_data = data[data['Grade'] == 'K']

# Calculate total number of Kindergarten seats
total_kindergarten_seats = kindergarten_data['Total Seats'].sum()

# Calculate  percentage of seats reserved for language programs 
kindergarten_data['Language Seat Percentage'] = (kindergarten_data['Total Seats'] / total_kindergarten_seats) * 100

# 1. Schools exceeding 35% of Kindergarten seats for language programs
schools_exceeding_35_percent = kindergarten_data[kindergarten_data['Language Seat Percentage'] > 35]

# 2. Schools with high demand (Requests per seat > 1.0) for language programs
high_demand_language_programs = kindergarten_data[kindergarten_data['Requests per seat'] > 1.0]

# 3. Schools with low demand (Requests per seat < 0.4) for language programs
low_demand_language_programs = kindergarten_data[kindergarten_data['Requests per seat'] < 0.4]

# Output the names of schools that meet the criteria
print("\nSchools exceeding 35% of Kindergarten seats for language programs:")
print(schools_exceeding_35_percent[['School Name', 'Program Name', 'Language Seat Percentage']])

print("\nSchools with high demand for language programs (Requests per seat > 1.0):")
print("\The below schools should likely expand their language program")
print(high_demand_language_programs[['School Name', 'Program Name', 'Requests per seat']])


print("\nSchools with low demand for language programs (Requests per seat < 0.4):")
print("\The below schools can likely remove their language program")
print(low_demand_language_programs[['School Name', 'Program Name', 'Requests per seat']])


# 4. Calculate the MAXIMUM number of language pathway seats that can be added
max_language_seats_allowed = 0.35 * total_kindergarten_seats  # 35% of total Kindergarten seats
current_language_seats = kindergarten_data[kindergarten_data['Program Code'] != 'GE']['Total Seats'].sum()

# Calculate how many additional language seats can be added
additional_language_seats = max_language_seats_allowed - current_language_seats
print(f"\nCurrent language seats: {current_language_seats}")
print(f"Maximum allowed language seats (35% of total): {max_language_seats_allowed}")
print(f"Additional language seats that can be added: {additional_language_seats}")

# 5. Sort high-demand programs by Requests per seat in descending order to prioritize adding seats
sorted_high_demand = high_demand_language_programs.sort_values(by='Requests per seat', ascending=False)

# 6. Calculate and update the number of language pathway seats to be added
remaining_language_seats_to_add = additional_language_seats
expanded_programs = []

for index, row in sorted_high_demand.iterrows():
    if remaining_language_seats_to_add > 0:
        # Determine how many seats to add based on demand
        # Add as many seats as possible without exceeding the remaining available seats
        add_seats = min(row['Total Seats'], remaining_language_seats_to_add)
        
        # Update the number of remaining seats to be added
        remaining_language_seats_to_add -= add_seats
        
        # Add the updated program to the list of expanded programs
        expanded_programs.append((row['School Name'], row['Program Name'], add_seats))
        
        if remaining_language_seats_to_add <= 0:
            break

# 7. Output the updated language programs with added seats
print("\nUpdated language programs with added seats:")
for program in expanded_programs:
    print(f"School: {program[0]}, Program: {program[1]}, Added Seats: {program[2]}")

# 8. Ensure 65% of seats are for General Education
total_general_education_seats = total_kindergarten_seats - current_language_seats - sum([p[2] for p in expanded_programs])
if total_general_education_seats / total_kindergarten_seats >= 0.65:
    print("\nThe allocation of seats ensures at least 65% for General Education.")
else:
    print("\nWarning: General Education seats are below 65% after allocation.")

# Optionally, print the high-demand programs and schools for further inspection
print("\nHigh demand language programs:")
print(sorted_high_demand[['School Name', 'Program Name', 'Requests per seat']])









Schools exceeding 35% of Kindergarten seats for language programs:
Empty DataFrame
Columns: [School Name, Program Name, Language Seat Percentage]
Index: []

Schools with high demand for language programs (Requests per seat > 1.0):
\The below schools should likely expand their language program
                School Name                                  Program Name  \
1152               Alamo ES                         GE: General Education   
1154            Alvarado ES                         GE: General Education   
1155            Alvarado ES                  MS: SDC - Extensive Services   
1156            Alvarado ES            SE: Spanish Immersion (Non-Native)   
1157            Alvarado ES      SN: Spanish Immersion (Spanish Speakers)   
...                     ...                                           ...   
1305   Visitacion Valley ES                         GE: General Education   
1306  Chin (John Yehall) ES                         GE: General Education   
1307        